In [347]:
import networkx as nx
import numpy as np
from tqdm import tqdm
from itertools import combinations

In [389]:
with open("../data/16.txt") as f:
    data = [line.rstrip() for line in f.readlines()]

nr = len(data)
nc = len(data[0])
data = np.array([dd for d in data for dd in d]).reshape((nr,nc))
assert nr == nc

In [390]:
def get_idx(r,c,nc=nc):
    return (r*nc)+c

def invert_idx(idx,nc,nr):
    return (idx//nr,idx%nc)

def take_step(i,step,data=data):
    nr,nc = data.shape
    move = (i[0]+step[0],i[1]+step[1])
    if move[0]>=0 and move[1]>=0 and move[0]<nc and move[1]<nr and data[move]!="#":
        
        return move

In [391]:
g = nx.Graph()

# add all the nodes
# with z dimension representing different facings
i=0
for r in range(nr):
    for c in range(nc):
        assert i == get_idx(r,c,nc)
        [g.add_node(f"{i}_{z}") for z in range(4)]
        i+=1


# directions = [
#     (-1,0), 
#     (1,0),
#     (0,1),
#     (0,-1)
# ]

direction_lkup = {
    
    (0,1):0, # east
    (-1,0):1, # north
    (0,-1):2, # west
    (1,0):3 # south
}

reverse_direction_lkup = {v:k for k,v in direction_lkup.items()}



In [392]:

# add xy edges for each position
for r in range(nr):
    for c in range(nc):
        if data[r,c] != "#":
            for z in range(4):
                step = take_step((r,c),reverse_direction_lkup[z],data)
                if step:
                    g.add_edge(f"{get_idx(r,c)}_{z}",f"{get_idx(*step)}_{z}",weight=1)

for r in range(nr):
    for c in range(nc):
        if data[r,c] != "#":
            [g.add_edge(f"{get_idx(r,c)}_{z}",f"{get_idx(r,c)}_{(z+1)%4}",weight=1000) for z in range(4)]


In [393]:
start = np.where(data=="S")
start = get_idx(start[0][0],start[1][0],nc)

end = np.where(data=="E")
end = get_idx(end[0][0],end[1][0],nc)


In [394]:
shortest = nx.shortest_path_length(g,target=f"{int(start)}_0",weight="weight")

In [395]:
shortest_paths = [shortest[f"{int(end)}_{z}"] for z in range(4)]


In [396]:
min(shortest_paths)

82460